In [20]:
# import libraries
%matplotlib inline
import numpy as np
#import csv
import matplotlib.pyplot as plt
import pandas as pd
import glob
import ulmo
import os
import scipy.spatial

see: http://stackoverflow.com/questions/35296935/python-calculate-lots-of-distances-quickly

In [3]:
# subset the GHCN station list with the list of available stations
currentstations = ulmo.ncdc.ghcn_daily.get_stations(start_year=1985, end_year = 2016, elements = ['TMIN', 'TMAX', 'AWND'], as_dataframe=True)
currentGHCNstations = np.intersect1d(currentstations.id, ghcn.index.values) #ghcn['GHCN ID'].values)
ghcnSubset = ghcn.set_index('GHCN ID').loc[currentstations.id.values]

# at this point, ghcn must have the station id set as the index 

In [4]:
ghcn = ghcn.set_index('GHCN ID').loc[currentstations.id.values]

In [56]:
ghcn = pd.DataFrame(ulmo.ncdc.gsod.get_stations(country = 'US', start = '1985-01-01')).T

# append the brightness index 
BI = np.load('data/brightnessGSOD.npy')
ghcn['Brightness'] = BI

In [46]:
ghcn.shape

(6122, 11)

In [59]:
ghcn

,USAF,WBAN,begin,country,elevation,end,icao,latitude,longitude,name,state,Brightness
423630-99999,423630,99999,2013-02-06,US,37,2016-06-18,,28.16,-89.22,MISSISSIPPI CANYON OIL PLATFORM,LA,11.0
619760-99999,619760,99999,1973-01-01,US,13,2016-08-22,,-15.883,54.517,SERGE-FROLOW (ILE TROMELIN),,3.0
621010-99999,621010,99999,2008-07-21,US,-999,2008-07-21,,50.6,-2.933,MOORED BUOY,,3.0
621110-99999,621110,99999,2004-11-18,US,-999,2009-12-01,,58.9,-0.2,MOORED BUOY,,3.0
621130-99999,621130,99999,2004-07-26,US,-999,2004-07-26,,58.4,0.3,MOORED BUOY,,5.0
621140-99999,621140,99999,2009-06-17,US,-999,2010-01-03,,58.7,1.1,MOORED BUOY,,5.0
621160-99999,621160,99999,2004-08-29,US,-999,2004-08-29,,58.1,1.8,MOORED BUOY,,4.0
621170-99999,621170,99999,2004-07-26,US,-999,2011-08-09,,57.9,0.1,MOORED BUOY,,4.0
621180-99999,621180,99999,2005-07-01,US,-999,2009-11-19,,57.7,0.9,MOORED BUOY,,11.0
621220-99999,621220,99999,2004-08-01,US,-999,2009-02-09,,56.5,2.017,MOORED BUOY,,4.0


In [5]:
# compute distances between all stations
#tree = scipy.spatial.cKDTree(giss[['lon', 'lat']].values, leafsize=100)
# query the closest point 
#closestInd = tree.query(giss[['lon', 'lat']].values[11,:], k =2, distance_upper_bound=6)[1][1]

In [61]:
atlas = pd.read_csv('data/sampleAtlas.csv') # derived  from http://www.naturalearthdata.com/downloads/10m-cultural-vectors/10m-populated-places/
atlas = pd.read_csv('data/world_cities.csv')
tree = scipy.spatial.cKDTree(ghcn[['longitude', 'latitude']].values, leafsize=100)

atlas = atlas[atlas['pop']> 500000]

In [63]:
frames = []

for i in range(0, atlas.shape[0]): 
    lat = atlas.iloc[i]['lat']
    lon = atlas.iloc[i]['lng']
    city = atlas.iloc[i]['city']
    # look up the closest k stations within the distance_upper_bound
    closeststations = tree.query([lon,lat], k =15, distance_upper_bound=.5)
    # Make sure the closest stations are within a finite distance
    closestInds = closeststations[1][~np.isinf(closeststations[0])]
    # If there are no stations, skip. Otherwise, 
    if closestInds.shape[0] != 0: 
        # Urban stations are the brightest; return 
        urban = ghcn.Brightness[closestInds].argmax()
        rural = ghcn.Brightness[closestInds].argmin()
        if urban != rural: 
            frames.append([city, ghcn.iloc[closestInds].loc[urban].name, #['GHCN ID'],
                           ghcn.iloc[closestInds].loc[urban]['latitude'],
                           ghcn.iloc[closestInds].loc[urban]['longitude'],
                           ghcn.Brightness.iloc[closestInds].max(), 

                           ghcn.iloc[closestInds].loc[rural].name, #['GHCN ID'],
                           ghcn.iloc[closestInds].loc[rural]['latitude'],
                           ghcn.iloc[closestInds].loc[rural]['longitude'],

                           ghcn.Brightness.iloc[closestInds].min()])

pairs = pd.DataFrame(frames, columns = ['City', 'Urban station', 
                                      'Urban Lat', 'Urban Lon','Urban brightness', 
                                      'Rural station', 
                                      'Rural Lat', 'Rural Lon','Rural brightness'])
pairs.to_csv('GSODpairedstations.csv')

In [64]:
pairs.shape

(71, 9)

In [10]:
pairs[pairs['Urban station'].str.contains('US')].shape

(71, 9)

In [ ]:
# Save out the several station matches 
frames = []

for i in range(0, atlas.shape[0]): 
    lat = atlas.iloc[i]['lat']
    lon = atlas.iloc[i]['lng']
    city = atlas.iloc[i]['city']
    # look up the closest k stations within the distance_upper_bound
    closeststations = tree.query([lon,lat], k =15, distance_upper_bound=.5)
    # Make sure the closest stations are within a finite distance
    closestInds = closeststations[1][~np.isinf(closeststations[0])]
    # If there are no stations, skip. Otherwise, 
    if closestInds.shape[0] != 0: 
        # Urban stations are the brightest; return 
        urban = ghcn.Brightness[closestInds].argmax()
        rural = ghcn.Brightness[closestInds].argmin()
        if urban != rural: 
            frames.append([city, ghcn.iloc[closestInds].loc[urban].name, #['GHCN ID'],
                           ghcn.iloc[closestInds].loc[urban]['lat'],
                           ghcn.iloc[closestInds].loc[urban]['lon'],
                           ghcn.Brightness.iloc[closestInds].max(), 

                           ghcn.iloc[closestInds].loc[rural].name, #['GHCN ID'],
                           ghcn.iloc[closestInds].loc[rural]['lat'],
                           ghcn.iloc[closestInds].loc[rural]['lon'],

                           ghcn.Brightness.iloc[closestInds].min()])

pairs = pd.DataFrame(frames, columns = ['City', 'Urban station', 
                                      'Urban Lat', 'Urban Lon','Urban brightness', 
                                      'Rural station', 
                                      'Rural Lat', 'Rural Lon','Rural brightness'])
pairs.to_csv('GHCNpairedstations.csv')

In [25]:
# calculate the brightness index for all stations
from osgeo import ogr, osr
import os
from shapely.geometry import Point
import shapely.geometry
import shapely.wkt
from cartopy.feature import ShapelyFeature
from cartopy.io.shapereader import Reader
import gdal

# rasterfile = 'data/F182013.v4/F182013.v4c_web.stable_lights.avg_vis.tif'
# layer = gdal.Open(rasterfile)
# gt =layer.GetGeoTransform()
# bands = layer.RasterCount
# src = layer.GetRasterBand(1)
#X = ghcnSubset['lon']
#Y = ghcnSubset['lat']

X = gsod['longitude'].values
Y = gsod['latitude'].values

In [54]:
#rasterfile = 'data/F182013.v4/F182013.v4c_web.stable_lights.avg_vis.tif'
rasterfile = 'data/F182013.v4/F182013.v4c_web.avg_vis.tif'
layer = gdal.Open(rasterfile)
gt =layer.GetGeoTransform()
bands = layer.RasterCount
src = layer.GetRasterBand(1)

BI = np.nan*np.ones(X.shape)
i = 0 
for x,y in zip(X,Y): 
    if x: 
        rasterx = int((x - gt[0]) / gt[1])
        rastery = int((y - gt[3]) / gt[5])
        BI[i] = src.ReadAsArray(rasterx,rastery, win_xsize=1, win_ysize=1)
    i = i+1

np.save('data/brightnessGSOD.npy',BI)
#np.save('brightnessGHCNsubset.npy', BI)
#BI = np.load('data/brightnessGHCN.npy')

In [55]:
BI.shape

(6122,)

In [49]:
BI.shape

(100747,)

In [50]:
BI = np.nan*np.ones(X.shape)
BI.shape

(6122,)